
# AWS EC2 インスタンスの使用

:ラベル: `sec_aws`

このセクションでは、すべてのライブラリを未加工の Linux マシンにインストールする方法を説明します。 :numref: `sec_sagemaker`で、Amazon SageMaker を使用する方法について説明しましたが、AWS では自分でインスタンスを構築する方がコストが安くなります。ウォークスルーには 3 つのステップが含まれます。
1.  AWS EC2 から GPU Linux インスタンスをリクエストします。
1.  CUDA をインストールします (または、CUDA がプリインストールされた Amazon マシンイメージを使用します)。
1. 本のコードを実行するための深層学習フレームワークとその他のライブラリをインストールします。

このプロセスは、多少の変更はありますが、他のインスタンス (および他のクラウド) にも適用されます。先に進む前に、AWS アカウントを作成する必要があります。詳細については、:numref: `sec_sagemaker`を参照してください。

##  EC2 インスタンスの作成と実行

AWS アカウントにログインした後、「EC2」(:numref: `fig_aws` ) をクリックして EC2 パネルに移動します。

![](../img/aws.png) :幅: `400px` :ラベル: `fig_aws`

 :numref: `fig_ec2` EC2 パネルを示します。

![](../img/ec2.png) :幅: `700px` :ラベル: `fig_ec2`

### プリセット位置

レイテンシを短縮するには、近くのデータセンターを選択します (例: 「Oregon」) ( :numref: `fig_ec2`の右上にある赤いボックスでマークされています)。中国にお住まいの場合は、ソウルや東京など、近くのアジア太平洋地域を選択できます。一部のデータセンターには GPU インスタンスがない場合があることに注意してください。

### 限界の増加

インスタンスを選択する前に、 :numref: `fig_ec2`に示すように、左側のバーの「制限」ラベルをクリックして数量制限があるかどうかを確認してください。 :numref: `fig_limits` 、そのような制限の例を示しています。現在、アカウントはリージョンごとに「p2.xlarge」インスタンスを開くことができません。 1 つ以上のインスタンスを開く必要がある場合は、[制限の増加をリクエスト] リンクをクリックして、インスタンス クォータの増加を申請します。通常、申請の処理には 1 営業日かかります。

![](../img/limits.png) :幅: `700px` :ラベル: `fig_limits`

### インスタンスの起動

次に、:numref: `fig_ec2`の赤枠で囲まれた「Launch Instance」ボタンをクリックしてインスタンスを起動します。

まず、適切な Amazon Machine Image (AMI) を選択します。 Ubuntu インスタンス (:numref: `fig_ubuntu` ) を選択します。

![](../img/ubuntu-new.png) :幅: `700px` :ラベル: `fig_ubuntu`

 EC2 には、選択できるさまざまなインスタンス構成が多数用意されています。これは初心者にとっては圧倒されるかもしれません。 :numref: `tab_ec2`は、さまざまな適切なマシンがリストされます。

 :さまざまな EC2 インスタンス タイプ :label: `tab_ec2`

 |名前 | GPU |メモ | |------|---------------|---------------------------- ---| | g2 |グリッドK520 |古代 | | p2 |ケプラー K80 |古いがスポットとして安いことが多い | | g3 |マクスウェル M60 |良いトレードオフ | | p3 |ボルタV100 | FP16 の高性能 | | p4 |アンペア A100 |大規模なトレーニング向けの高性能 | | g4 |チューリング T4 |推論最適化された FP16/INT8 |

これらすべてのサーバーには、使用される GPU の数を示す複数の種類があります。たとえば、p2.xlarge には 1 つの GPU が搭載されており、p2.16xlarge には 16 つの GPU とそれ以上のメモリが搭載されています。詳細については、 [AWS EC2 ドキュメント](https://aws.amazon.com/ec2/instance-types/)または[概要ページを](https://www.ec2instances.info)参照してください。説明のためには、 p2.xlarge で十分です ( :numref: `fig_p2x`の赤いボックス内にマークされています)。

![](../img/p2x.png) :幅: `700px` :ラベル: `fig_p2x`

適切なドライバーと GPU 対応の深層学習フレームワークを備えた GPU 対応のインスタンスを使用する必要があることに注意してください。そうしないと、GPU を使用するメリットが得られません。

インスタンスへのアクセスに使用するキー ペアの選択に進みます。キーペアがない場合は、:numref: `fig_keypair`の「新しいキーペアの作成」をクリックしてキーペアを生成します。その後、以前に生成したキー ペアを選択できます。新しいキー ペアを生成した場合は、必ずキー ペアをダウンロードし、安全な場所に保存してください。これはサーバーに SSH で接続する唯一の方法です。

![](../img/keypair.png) :幅: `500px` :ラベル: `fig_keypair`

この例では、「ネットワーク設定」のデフォルト設定をそのまま使用します（サブネットやセキュリティグループなどの項目を設定するには、「編集」ボタンをクリックします）。デフォルトのハードディスク サイズを 64 GB (:numref: `fig_disk` ) に増やすだけです。 CUDA 自体がすでに 4 GB を占有していることに注意してください。

![](../img/disk.png) :幅: `700px` :ラベル: `fig_disk`

 「Launch Instance」をクリックすると、作成したインスタンスが起動します。 :numref: `fig_launching`に表示されているインスタンス ID をクリックして、このインスタンスのステータスを表示します。

![](../img/launching.png) :幅: `700px` :ラベル: `fig_launching`

### インスタンスへの接続

:numref: `fig_connect`に示すように、インスタンスの状態が緑色に変わったら、インスタンスを右クリックして`Connect`を選択し、インスタンスのアクセス方法を表示します。 

![](../img/connect.png) :幅: `700px` :ラベル: `fig_connect`

これが新しいキーの場合、SSH が機能するためには公開されてはなりません。 `D2L_key.pem`を保存するフォルダーに移動し、次のコマンドを実行してキーを非公開にします。

![](../img/chmod.png) :幅: `400px` :ラベル: `fig_chmod`

次に、:numref: `fig_chmod`の下の赤いボックスにある ssh コマンドをコピーし、コマンド ラインに貼り付けます。

コマンドラインで「接続を続行してもよろしいですか (はい/いいえ)」というプロンプトが表示されたら、「yes」と入力して Enter キーを押し、インスタンスにログインします。

これでサーバーの準備が整いました。

##  CUDAのインストール

CUDA をインストールする前に、必ず最新のドライバーでインスタンスを更新してください。

ここでは CUDA 10.1 をダウンロードします。 NVIDIA の[公式リポジトリ](https://developer.nvidia.com/cuda-toolkit-archive)にアクセスして、 :numref: `fig_cuda`に示されているダウンロード リンクを見つけます。 

![](../img/cuda101.png) :幅: `500px` :ラベル: `fig_cuda`

手順をコピーして端末に貼り付け、CUDA 10.1 をインストールします。

プログラムをインストールした後、次のコマンドを実行して GPU を表示します。

最後に、他のライブラリが見つけやすくするために、ライブラリ パスに CUDA を追加します。

## コードを実行するためのライブラリのインストール

この本のコードを実行するには、:ref: EC2 インスタンス上の Linux ユーザー向けの`chap_installation`手順に従い、リモート Linux サーバーで作業するための次のヒントを使用してください。
-  Miniconda インストール ページで bash スクリプトをダウンロードするには、ダウンロード リンクを右クリックして [リンク アドレスのコピー] を選択し、 `wget [copied link address]`を実行します。
-  `~/miniconda3/bin/conda init`を実行した後、現在のシェルを閉じて再度開く代わりに、 `source ~/.bashrc`を実行できます。

##  Jupyter Notebook をリモートで実行する

Jupyter Notebook をリモートで実行するには、SSH ポート転送を使用する必要があります。結局のところ、クラウド上のサーバーにはモニターやキーボードがありません。このためには、次のようにデスクトップ (またはラップトップ) からサーバーにログインします。

次に、EC2 インスタンス上のこの本のダウンロードされたコードの場所に移動し、次を実行します。

 :numref: `fig_jupyter` Jupyter Notebook を実行した後に考えられる出力を示します。最後の行はポート 8888 の URL です。 

![](../img/jupyter.png) :幅: `700px` :ラベル: `fig_jupyter`

ポート 8889 へのポート転送を使用したため、 :numref: `fig_jupyter`の赤いボックスの最後の行をコピーし、URL の「8888」を「8889」に置き換えて、ローカル ブラウザで開きます。

## 使用されていないインスタンスを閉じる

クラウド サービスは使用時間に応じて課金されるため、使用されていないインスタンスは閉じておく必要があります。代替手段もあることに注意してください。
- インスタンスを「停止」すると、インスタンスを再度開始できるようになります。これは、通常のサーバーの電源を切るのと似ています。ただし、停止したインスタンスでも、保持されているハードディスク容量に対して少額が請求されます。
- インスタンスを「終了」すると、インスタンスに関連付けられたすべてのデータが削除されます。これにはディスクが含まれるため、再度起動することはできません。これは、将来必要なくなることがわかっている場合にのみ行ってください。

このインスタンスをさらに多くのインスタンスのテンプレートとして使用する場合は、:numref: `fig_connect`の例を右クリックし、「イメージ」 $\rightarrow$ 「作成」を選択してインスタンスのイメージを作成します。これが完了したら、「インスタンスの状態」 $\rightarrow$ 「終了」を選択してインスタンスを終了します。次回このインスタンスを使用する場合は、このセクションの手順に従って、保存されたイメージに基づいてインスタンスを作成できます。唯一の違いは、 :numref: `fig_ubuntu`に示されている「1. AMI の選択」で、左側の「My AMI」オプションを使用して保存したイメージを選択する必要があることです。作成されたインスタンスは、イメージ ハードディスクに保存されている情報を保持します。たとえば、CUDA やその他のランタイム環境を再インストールする必要はありません。

## まとめ
- 独自のコンピューターを購入して構築することなく、オンデマンドでインスタンスを起動および停止できます。
-  GPU 対応の深層学習フレームワークを使用する前に、CUDA をインストールする必要があります。
- ポート転送を使用して、リモート サーバー上で Jupyter Notebook を実行できます。

## 演習
1. クラウドは利便性を提供しますが、その費用は安くありません。[スポット インスタンスを](https://aws.amazon.com/ec2/spot/)起動してコストを削減する方法を確認してください。
1. さまざまな GPU サーバーを試してください。彼らはどれくらい速いですか？
1. マルチ GPU サーバーを試してみましょう。物事をどれだけうまくスケールアップできるでしょうか?

[ディスカッション](https://discuss.d2l.ai/t/423)
